In [8]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision import models

In [5]:
import os
import shutil

# Original paths
src_fire = r'C:\Users\ahmed\Documents\Python Scripts\Realtime Flame Detection (CNN)\fire_dataset\fire_images'
src_non_fire = r'C:\Users\ahmed\Documents\Python Scripts\Realtime Flame Detection (CNN)\fire_dataset\non_fire_images'

# New structure
dst_base = 'fire_dataset_cleaned'
os.makedirs(dst_base, exist_ok=True)

for category, src in [('fire', src_fire), ('non_fire', src_non_fire)]:
    dst = os.path.join(dst_base, category)
    os.makedirs(dst, exist_ok=True)
    for file in os.listdir(src):
        shutil.copy(os.path.join(src, file), os.path.join(dst, file))


In [6]:
# Config
data_dir = 'fire_dataset_cleaned'
model_save_path = 'flame_detector.pth'
batch_size = 32
num_epochs = 5
learning_rate = 0.001
img_size = 224

In [9]:
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # mean
                         [0.229, 0.224, 0.225])  # std
])


dataset = ImageFolder(data_dir, transform=transform)


train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.last_channel, 2)  # Fire / No fire
model = model.to(device)

C:\Users\ahmed\anaconda3\envs\neuralnetwork\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\ahmed\anaconda3\envs\neuralnetwork\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\ahmed/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 25.0MB/s]


In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [12]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()

    acc = correct / len(train_ds)
    print(f"[Epoch {epoch+1}] Loss: {running_loss:.4f}, Accuracy: {acc:.4f}")

[Epoch 1] Loss: 4.5220, Accuracy: 0.9274
[Epoch 2] Loss: 2.3903, Accuracy: 0.9687
[Epoch 3] Loss: 1.2208, Accuracy: 0.9825
[Epoch 4] Loss: 1.6888, Accuracy: 0.9712
[Epoch 5] Loss: 0.6529, Accuracy: 0.9912


In [13]:
torch.save(model.state_dict(), model_save_path)